In [623]:
from __future__ import annotations

import pandas as pd
from typing import Type, List
import json

## Generating enum

In [624]:
def enum_name(s: str):
    return s.replace("-", "_").upper()

def method_name(s: str):
    return s.replace("-", "_")

def getter_name(s: str):
    return "get " + method_name(s) + "()"

def new_line(s:str):
    return s + "\n"

def comma(s: str):
    return s + ','

def space(s: str):
    return s + ' '

def quotes(s: str):
    return f'"{s}"'

def tab(s: str):
    return "    " + s

def collection_name(singular: str):
    def _collection_name(s: str):
        return singular + '.' + s
    return _collection_name

def accessor(array_name: str, property_name: str, data_class: str):
    return (
        f'    {getter_name(property_name)} {{\n'
        f'        return {data_class}Configuration.{array_name}[this._id]\n'
        f'    }}\n'
    )

def id_accessor():
    return (
        f'    {getter_name("id")} {{\n'
        f'        return this._id\n'
        f'    }}\n'
    )

In [625]:
class DataEnum:
    def __init__(self, name_singular, name_plural, axis_name) -> None:
        self.name_singular = name_singular
        self.name_plural = name_plural
        self.axis_name = axis_name
        self.is_id = False

    def extract_definition_from_table(self, data: pd.DataFrame):
        return (
            f'export const enum {self.name_singular} ' '{\n'
            f'{data[self._axis_name()].apply(enum_name).drop_duplicates(keep="first").apply(comma).apply(new_line).apply(tab).sum()}'
            '}\n'
        )


    def enum_type_symbol(self):
        return f'{enum_name(self.name_singular)}'
    def mapping_type_symbol(self, id_enum:DataEnum):
        return f'Record<{id_enum.name_singular}, {self.name_singular}>'
    def mapping_string_type_symbol(self, id_enum:DataEnum):
        return f'Record<{id_enum.name_singular}, string>'
    def array_type_symbol(self):
        return f'{self.name_singular}[]'
    def array_strings_type_symbol(self):
        return f'Record<{self.enum_type_symbol()}, string>'
    def dict_type_symbol(self):
        return f'Record<string, {self.enum_type_symbol()}>'


    def array_symbol(self):
        return f'{enum_name(self.axis_name)}'
    def array_strings_symbol(self):
        return f'{enum_name(self.axis_name)}_TO_STRING'
    def dict_symbol(self):
        return f'{enum_name(self.axis_name)}_FROM_STRING'


    def mapping_symbol(self, id_enum: DataEnum):
        return f'{enum_name(id_enum.name_singular)}_{enum_name(self.axis_name)}'
    def mapping_string_symbol(self, id_enum: DataEnum):
        return f'{enum_name(id_enum.name_singular)}_{enum_name(self.axis_name)}_STRING'

    def _axis_name(self):
        if self.is_id:
            return "id"
        return self.axis_name

    def extract_python_list_of_strings(self, data: pd.DataFrame):
        return data[self._axis_name()].drop_duplicates(keep="first").values

    def extract_python_list_of_enum_names(self, data: pd.DataFrame):
        return data[self._axis_name()].apply(enum_name).drop_duplicates(keep="first").apply(collection_name(self.name_singular)).values

    def extract_array_from_table(self, data: pd.DataFrame):
        return f'    static {self.array_symbol()} : {self.array_type_symbol()} = [{data[self._axis_name()].apply(enum_name).drop_duplicates(keep="first").apply(collection_name(self.name_singular)).apply(comma).apply(space).sum()}]\n'
    def extract_string_array_from_table(self, data: pd.DataFrame):
        return f'    static {self.array_strings_symbol()} : {self.array_strings_type_symbol()} = [{data[self._axis_name()].apply(quotes).drop_duplicates(keep="first").apply(comma).apply(space).sum()}]\n'


    def extract_inverse_mapping(self, data: pd.DataFrame):
        data_string = ", ".join([f'"{k}": {v}' for k, v in zip(self.extract_python_list_of_strings(data), self.extract_python_list_of_enum_names(data))])
        return f'    static {self.dict_symbol()} : {self.dict_type_symbol()} = {{ {data_string} }}\n'

    def extract_mapping_string(self, data: pd.DataFrame, id_enum: DataEnum):
        return f'    static {self.mapping_string_symbol(id_enum)} : {self.mapping_string_type_symbol(id_enum)} = [{data[self._axis_name()].apply(str).apply(quotes).apply(comma).apply(space).sum()}]\n'

    def extract_mapping(self, data: pd.DataFrame, id_enum: DataEnum):
        return f'    static {self.mapping_symbol(id_enum)} : {self.mapping_type_symbol(id_enum)} = [{data[self._axis_name()].apply(enum_name).apply(collection_name(self.name_singular)).apply(comma).apply(space).sum()}]\n'

    def getter_string(self, id_enum:DataEnum, table_name: str):
        return accessor(f'{self.mapping_string_symbol(id_enum)}', self._axis_name() + "_string", table_name)

    def getter(self, id_enum: DataEnum, table_name: str):
        if (id_enum == self):
            return id_accessor()
        return accessor(f'{self.mapping_symbol(id_enum)}', self._axis_name(), table_name)

### Export data from table


In [626]:
def number_series_to_array(s: pd.Series, id_enum: DataEnum, axis_name) -> str:
    return f'    static {id_enum.name_singular}_{enum_name(axis_name)} : Record<{id_enum.name_singular}, number> = [{s.apply(str).apply(comma).apply(space).sum()}]'

def string_series_to_array(s: pd.Series, id_enum: DataEnum, axis_name) -> str:
    return f'    static {id_enum.name_singular}_{enum_name(axis_name)} : Record<{id_enum.name_singular}, string> = [{s.apply(quotes).apply(comma).apply(space).sum()}]'

In [627]:
def extract_data_from(table_name: str, d: pd.DataFrame, id_enum:DataEnum, enums: List[DataEnum], string_columns: List[str], number_columns: List[str]):
    id_enum.is_id = True

    configuration_string = ""
    accessor_string = ""
    interface_string = ""

    enum_definition = id_enum.extract_definition_from_table(d)

    configuration_string += id_enum.extract_array_from_table(d)
    configuration_string += id_enum.extract_inverse_mapping(d)
    configuration_string += id_enum.extract_string_array_from_table(d) + "\n    // ENUMS: \n\n"

    for item in [id_enum] + enums:
        configuration_string += item.extract_mapping(d, id_enum)
        configuration_string += item.extract_mapping_string(d, id_enum)
        accessor_string += item.getter(id_enum, table_name)
        accessor_string += item.getter_string(id_enum, table_name)
        interface_string += f'    readonly {method_name(item._axis_name())} : {item.enum_type_symbol()}\n';

    configuration_string += "\n    // Numbers: \n"

    for item in number_columns:
        configuration_string = "\n".join([configuration_string, number_series_to_array(d[item], id_enum, item)]);
        accessor_string = "".join([accessor_string, accessor(f'{id_enum.name_singular}_{enum_name(item)}', item, table_name)]);
        interface_string += f'    readonly {method_name(item)} : number\n';

    configuration_string += "\n\n    // Strings: \n"

    for item in string_columns:
        configuration_string = "\n".join([configuration_string, string_series_to_array(d[item], id_enum, item)])
        accessor_string = "".join([accessor_string, accessor(f'{id_enum.name_singular}_{enum_name(item)}', item, table_name)])
        interface_string += f'    readonly {method_name(item)} : string\n'

    interface_string = (
        f'export interface {table_name}Data {{\n'
        f'{interface_string}'
        f'}}\n\n'
    )

    configuration_string = (
        f'export class {table_name}Configuration {{\n'
        f'{configuration_string}\n'
        f'}}\n'
    )

    accessor_string = (
        f'\n'
        f'export class {table_name} implements {table_name}Data {{\n'
        f'    private _id: {id_enum.enum_type_symbol()}\n'
        f'    constructor(id: {id_enum.enum_type_symbol()}) {{\n'
        f'        if (!(id in {table_name}Configuration.{id_enum.array_symbol()})) {{\n'
        f'            throw new Error(`Invalid {table_name} id: ${{id}}`);\n'
        f'        }}\n'
        f'        this._id = id\n'
        f'    }}\n'
        f'    static from_string(id: string) {{\n'
        f'        if (!(id in {table_name}Configuration.{id_enum.dict_symbol()})) {{\n'
        f'            throw new Error(`Invalid {table_name} id: ${{id}}`);\n'
        f'        }}\n'
        f'        return new {table_name}({table_name}Configuration.{id_enum.dict_symbol()}[id])\n'
        f'    }}\n'
        f'{accessor_string}'
        f'}}\n'
    )

    id_enum.is_id = False

    return enum_definition, interface_string, configuration_string, accessor_string

### Export to files

#### Defining enums

In [628]:
enum_MATERIAL_ID = DataEnum("MATERIAL", "MATERIALS", "material")
enum_SECONDARY_MATERIAL_ID = DataEnum("MATERIAL", "MATERIALS", "secondary-material")
enum_MATERIAL_CATEGORY = DataEnum("MATERIAL_CATEGORY", "MATERIAL_CATEGORIES", "category")
enum_EQUIP_SLOT = DataEnum("EQUIP_SLOT", "EQUIP_SLOTS", "slot")
enum_WEAPON_ID = DataEnum("WEAPON", "WEAPONS", "weapon")
enum_ARMOUR_ID = DataEnum("ARMOUR", "ARMOURS", "armour")
enum_IMPACT = DataEnum("IMPACT_TYPE", "IMPACT_TYPES", "impact")

#### Materials

In [629]:
materials = pd.read_csv("tables/materials.csv");
number_columns = ['cutting-power', 'density', 'cutting-protection',
        'blunt-protection', 'penentration-protection', 'magic-power']
string_columns = ['name']
enum_columns = ['tag', 'category']

list_of_data =[]

list_of_data.append(extract_data_from(
    "Material",
    materials,
    enum_MATERIAL_ID,
    [enum_MATERIAL_CATEGORY],
    string_columns,
    number_columns
))

list_of_data.append(extract_data_from(
    "MaterialCategory",
    pd.read_csv("tables/material-categories.csv"),
    enum_MATERIAL_CATEGORY,
    [],
    ["name"],
    []
))

list_of_data.append(extract_data_from(
    "EquipSlot",
    pd.read_csv("tables/slots.csv"),
    enum_EQUIP_SLOT,
    [],
    ["name"],
    []
))

list_of_data.append(extract_data_from(
    "ImpactType",
    pd.read_csv("tables/impact.csv"),
    enum_IMPACT,
    [],
    ["name"],
    []
))

list_of_data.append(extract_data_from(
    "Weapon",
    pd.read_csv("tables/weapon.csv"),
    enum_WEAPON_ID,
    [enum_MATERIAL_ID, enum_SECONDARY_MATERIAL_ID, enum_IMPACT],
    ["name"],
    ["power","size","length","craftable"]
))

list_of_data.append(extract_data_from(
    "Armour",
    pd.read_csv("tables/armour.csv"),
    enum_ARMOUR_ID,
    [enum_MATERIAL_ID, enum_SECONDARY_MATERIAL_ID, enum_EQUIP_SLOT,],
    ["name"],
    ["magic-power","thickness","size","secondary-size","craftable"]
))

with open("src/content.ts", "w") as file:
    for i in range(4):
        for item in list_of_data:
            print(item[i], file=file)